<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 1.0 Visualize Synthetic Data with Bounding Boxes and Labels

In this notebook, you'll explore the synthetic data created with Replicator and visualize it.

**[1.1 Learning Objectives](#1.1-Learning-Objectives)<br>**
**[1.2 Setup for Visualization](#1.2-Setup-for-Visualization)<br>**
**[1.3 Write Visualization Functions](#1.3-Write-Visualization-Functions)<br>**
**[1.4 Load Synthetic Data and Visualize](#1.4-Load-Synthetic-Data-and-Visualize)<br>**

---
## 1.1 Learning Objectives

At this point, we have generated our synthetic data. We have three formats of data, and in this notebook we will combine the information from all three to visualize the data in full.
We have:
- The RGB image 
- The bounding box data with coordinates and a key to the fruit value
- Key data to correlate the bounding boxes to the actual fruit names

We will visualize our raw data with the ground truth labels.

<center><video controls src="https://dli-lms.s3.amazonaws.com/assets/s-ov-10-v1/DLI_part_4.mp4" width=800 ></center>

---
## 1.2 Setup for Visualization

In [ ]:
import os
import json

import hashlib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

We can navigate to the data we generated by opening up a terminal in our JupyterLab window. Press the "+" button in the top left to access a new terminal. Our data generation script defaults to `/dli/task/data/fruit_data_$DATE`. For now, we have the data directory set to an example dataset you may choose to use. Go ahead and pick a number of the image you would like to use.

In [ ]:
data_dir = "/dli/task/data/fruit_data"
out_dir = "/dli/task"
number = "26"

---
## 1.3 Write Visualization Functions

Sample helper visualization functions are provided in the [documentation](https://docs.omniverse.nvidia.com/prod_extensions/prod_extensions/ext_replicator/programmatic_visualization.html#helper-visualization-functions). 

Our first helper function, `data_to_colour`, takes in our data from a specific label ID and maps it to the proper color for the bounding box.  You can also find thes

In [ ]:
def data_to_colour(data):
    if isinstance(data, str):
        data = bytes(data, "utf-8")
    else:
        data = bytes(data)
    m = hashlib.sha256()
    m.update(data)
    key = int(m.hexdigest()[:8], 16)
    r = ((((key >> 0) & 0xFF) + 1) * 33) % 255
    g = ((((key >> 8) & 0xFF) + 1) * 33) % 255
    b = ((((key >> 16) & 0xFF) + 1) * 33) % 255

    inv_norm_i = 128 * (3.0 / (r + g + b))

    return (int(r * inv_norm_i) / 255, int(g * inv_norm_i) / 255, int(b * inv_norm_i) / 255)

The next helper function, `colorize_bbox_2d`, takes in the path to the RGB image for the background, the bounding box data, the labels, and the path to store the visualization. It outputs a colorized bounding box.

In [ ]:
def colorize_bbox_2d(rgb_path, data, id_to_labels, file_path):

    rgb_img = Image.open(rgb_path)
    colors = [data_to_colour(bbox["semanticId"]) for bbox in data]
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(rgb_img)
    for bbox_2d, color, index in zip(data, colors, range(len(data))):
        labels = id_to_labels[str(index)]
        rect = patches.Rectangle(
            xy=(bbox_2d["x_min"], bbox_2d["y_min"]),
            width=bbox_2d["x_max"] - bbox_2d["x_min"],
            height=bbox_2d["y_max"] - bbox_2d["y_min"],
            edgecolor=color,
            linewidth=2,
            label=labels,
            fill=False,
        )
        ax.add_patch(rect)

    plt.legend(loc="upper left")

    plt.savefig(file_path)


---
## 1.4 Load Synthetic Data and Visualize

Now we are ready to use these helper functions. We first get our desired image.

In [ ]:
rgb_path = data_dir
rgb = "rgb_"+number+".png" 
rgb_path = os.path.join(rgb_path, rgb)

Similarly, we load the bounding box data.

In [ ]:
npy_path = data_dir
bbox2d_tight_file_name = "bounding_box_2d_tight_"+number+".npy"
data = np.load(os.path.join(npy_path, bbox2d_tight_file_name))

Next, we load the labels corresponding to the image.

In [ ]:
json_path = data_dir
bbox2d_tight_labels_file_name = "bounding_box_2d_tight_labels_"+number+".json"
with open(os.path.join(json_path, bbox2d_tight_labels_file_name), "r") as json_data:
    bbox2d_tight_id_to_labels = json.load(json_data)

Finally, we can  call our function and see the labeled image!

In [ ]:
colorize_bbox_2d(rgb_path, data, bbox2d_tight_id_to_labels, os.path.join(out_dir, "bbox2d_tight.png"))

---
<h2 style="color:green;">Congratulations!</h2>

In this notebook, you have:
- Used Python functions to combine Replicator synthetic data outputs into a single visualization

Move on to the [Training notebook](2_train.ipynb)


<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>